# Multi-Label Neural Network Classifier for Reviews

We trained a multi-label neural network to predict whether a review is:

* is_ad (advertisement)

* is_relevant (relevant content)

* is_rant (rant/complaint)

using text embeddings, category embeddings, sentiment scores, and tabular features.

In [35]:
import os
import json
import time
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from pathlib import Path

# Import your API key from config
from config.config import OPENAI_API_KEY

from typing import List, Tuple, Iterable, Union
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler

import joblib

# 1. Data Preparation

Embeddings Generation:

* Used Qwen3-0.8B to generate dense vector embeddings for both:

* `category` (stored in categories_embeddings_all.parquet)

* `review_text` (stored in review_text_embeddings_all.parquet)

These embeddings capture semantic meaning of reviews and categories for downstream classification.

Combined parquet shards into unified datasets:

* `categories_embeddings_all.parquet`

* `review_text_embeddings_all.parquet`

* `review_sentiment_scores_all.parquet`

Merged these with `all_reviews_with_labels_and_features.parquet` on `review_id`.

Features used:

* Tabular: `rating`, `has_photo`, `category`, `average_score`, `average_rating`, `rating_discrepancy`, `sentiment_score`

* Embeddings: `category` + `review_text`

* Standardized numeric features (but not embeddings).

Sentiment Score Generation (ChatGPT):

* Used gpt-4o-mini to assign a numeric sentiment score in the range [-1.0, 1.0].

* Prompted the model with review text, enforcing strict JSON-only output (e.g., {"sentiment_score": -0.42}).

In [ ]:
all_reviews_labeled_df = pd.read_csv('all_reviews_with_labels_normalised.csv')

In [ ]:
all_reviews_labeled_df.head()

,review_text,rating,has_photo,author_name,user_review_count,business_name,category,source,review_id,comprehensive_review,is_ad,is_relevant,is_rant,is_legit
0,Love the convenience of this neighborhood carw...,4.0,False,Doug Schmidt,1.0,"Auto Spa Speedy Wash - Harvester, MO",['Car wash'],google,1001,"[Business] Auto Spa Speedy Wash - Harvester, M...",False,True,False,True
1,"2 bathrooms (for a large 2 story building), 1 ...",2.0,False,Duf Duftopia,1.0,Kmart,"['Discount store', 'Appliance store', 'Baby st...",google,1002,[Business] Kmart | [Category] ['Discount store...,True,True,True,False
2,My favorite pizza shop hands down!,5.0,False,Andrew Phillips,1.0,Papa’s Pizza,"['Pizza restaurant', 'Chicken wings restaurant...",google,1003,[Business] Papa’s Pizza | [Category] ['Pizza r...,False,True,False,True
3,BOTCHED INSTRUMENT REPAIR IS COSTING US HUNDRE...,1.0,False,Julie Heiland,1.0,The Music Place,['Musical instrument store'],google,1004,[Business] The Music Place | [Category] ['Musi...,False,True,True,False
4,Very unprofessional!!!!!,1.0,False,Alan Khasanov,1.0,Park Motor Cars Inc,['Used car dealer'],google,1005,[Business] Park Motor Cars Inc | [Category] ['...,False,True,True,False


In [ ]:
print("No. of reviews tagged as advertisements:", len(all_reviews_labeled_df[all_reviews_labeled_df['is_ad'] == True]))
print("No. of reviews tagged as irrelevant:", len(all_reviews_labeled_df[all_reviews_labeled_df['is_relevant'] == False]))
print("No. of reviews tagged as rants:", len(all_reviews_labeled_df[all_reviews_labeled_df['is_rant'] == True]))
print("No. of flagged reviews:", len(all_reviews_labeled_df[all_reviews_labeled_df['is_legit'] == False]))

No. of reviews tagged as advertisements: 391
No. of reviews tagged as irrelevant: 429
No. of reviews tagged as rants: 916
No. of flagged reviews: 1713


In [ ]:
all_reviews_labeled_df['comprehensive_review'][0]

"[Business] Auto Spa Speedy Wash - Harvester, MO | [Category] ['Car wash'] | [Rating] 4.0 | [Author] Doug Schmidt | [User Review Count] 1.0 | [Has Photo] no | [Source] google | [Review] Love the convenience of this neighborhood carwash."

### Embedding `review_text` and `categories`

We choose to embed `review_text` instead of `comprehensive_review` because `comprehensive_review` contains other fields from the dataset e.g. `categories`, `has_photo` etc. which might result in autocorrelation when we use these embeddings as predictors together with the other predictors. Embedding `review_text` by itself will ensure that we use pure semantic signals from the review as predictors, before combining them with the other predictors of the dataset. 

Given that many categories are sparse categories e.g. ['Discount store', 'Appliance store', 'Baby store', 'Bedding store', 'Clothing store', 'Department store', 'Electronics store', 'Home goods store', 'Shoe store', 'Toy store'] and others are hierarchical e.g. ['Bakery', 'Breakfast restaurant'], we want the model to be able to learn semantics between similar categories instead of as sprase binary predictors. Hence, we choose to embed `categories` as well, and use them as predictors for our model. 

In [ ]:
device_map="auto" if torch.cuda.is_available() else None

In [ ]:
# Using Qwen3-0.6B for embeddings for higher dimensionality
MODEL_ID = "Qwen/Qwen3-Embedding-0.6B"  # tiny, fast
MAX_LEN  = 256
BATCH    = 64                            # 0.6B can handle larger batches
SAVE_DIR = "./qwen3_embed_cache"
os.makedirs(SAVE_DIR, exist_ok=True)

# ---- load tokenizer/model on GPU if available ----
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

if torch.cuda.is_available():
    major_cc = torch.cuda.get_device_capability()[0]
    dtype = torch.bfloat16 if major_cc >= 8 else torch.float16
    device_map = "auto"
else:
    dtype = torch.float32
    device_map = None

model = AutoModel.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    device_map=device_map,
    trust_remote_code=True,
)
model.eval()

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

Qwen3Model(
  (embed_tokens): Embedding(151669, 1024)
  (layers): ModuleList(
    (0-27): 28 x Qwen3DecoderLayer(
      (self_attn): Qwen3Attention(
        (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
        (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
        (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
        (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
        (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
      )
      (mlp): Qwen3MLP(
        (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
        (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
        (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
      (post_attention_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
    )
  )
  (norm): Qwen3RMSNorm((102

In [ ]:
@torch.no_grad()
def _mean_pool(last_hidden_state: torch.Tensor, attention_mask: torch.Tensor) -> torch.Tensor:
    mask = attention_mask.unsqueeze(-1).type_as(last_hidden_state)        # [B,T,1]
    summed = (last_hidden_state * mask).sum(dim=1)                         # [B,H]
    counts = mask.sum(dim=1).clamp(min=1e-6)                               # [B,1]
    return summed / counts

@torch.no_grad()
def embed_texts(texts, max_len=MAX_LEN, batch_size=BATCH, normalize=True) -> np.ndarray:
    """Return np.ndarray [N, H] float32 (L2-normalized if normalize=True)."""
    embs = []
    N = len(texts)
    for i in trange(0, N, batch_size, desc="Embedding (Qwen3-Emb-0.6B)"):
        batch = [t if isinstance(t, str) and t.strip() else "" for t in texts[i:i+batch_size]]
        enc = tokenizer(batch, truncation=True, max_length=max_len, padding=True, return_tensors="pt")
        if torch.cuda.is_available():
            enc = {k: v.to(model.device) for k, v in enc.items()}
        out = model(**enc)
        pooled = _mean_pool(out.last_hidden_state, enc["attention_mask"])  # [B,H]
        pooled = pooled.float().cpu().numpy()
        if normalize:
            norms = np.linalg.norm(pooled, axis=1, keepdims=True) + 1e-12
            pooled = pooled / norms
        embs.append(pooled)
    return np.vstack(embs)

In [ ]:
def smoke_test(df, n=256):
    n = min(n, len(df))
    sample = df["review_text"].fillna("").iloc[:n].tolist()
    embs = embed_texts(sample, max_len=MAX_LEN, batch_size=BATCH, normalize=True)
    print("Smoke test OK — embeddings shape:", embs.shape)

In [ ]:
smoke_test(all_reviews_labeled_df, n=256)

Embedding (Qwen3-Emb-0.6B): 100%|██████████| 4/4 [03:47<00:00, 56.90s/it]

Smoke test OK — embeddings shape: (256, 1024)


We decided to use `Qwen3-Embedding-0.6B` for embedding after experimenting with `miniLM-L12-v2` and `Qwen-3-8B` because we wanted to balance high dimensionality of embeddings with a computationally efficient embedding process. `miniLM-L12-v2` produces embeddings with low dimensionality (384), while `Qwen-3-8B` produces high dimensionality (4096) but is computationally inefficient due to the high number of parameters. Hence, we decided on a light-weight model, `Qwen-3-Embedding-0.6B`, to improve efficiency and achieve higher dimensionality of 1024.

In [ ]:
import math
import numpy as np
import pandas as pd
from tqdm import trange
import pyarrow as pa
import pyarrow.parquet as pq

def embed_review_texts_batch(
    df: pd.DataFrame,
    out_parquet_path: str,
    batch_idx: int,
    chunk_rows: int = 2000,
    max_len: int = MAX_LEN,
    batch_size: int = BATCH,
    normalize: bool = True,
    store_dtype: str = "float32",
):
    """
    Embed ONE batch of reviews and append results to a Parquet file.

    Args:
        df : DataFrame with 'review_id' and 'review_text'
        out_parquet_path : Path to Parquet file
        batch_idx : Which batch index to embed (0-based)
        chunk_rows : How many rows per batch
    """
    assert "review_id" in df.columns and "review_text" in df.columns, \
        "DataFrame must have 'review_id' and 'review_text' columns."

    N = len(df)
    start = batch_idx * chunk_rows
    end = min(start + chunk_rows, N)
    if start >= N:
        raise IndexError(f"Batch {batch_idx} out of range. Max = {(N-1)//chunk_rows}")

    # Slice IN ORDER
    sl = df.iloc[start:end]
    texts = sl["review_text"].fillna("").tolist()
    ids   = sl["review_id"].tolist()

    # Embed
    embs = embed_texts(texts, max_len=max_len, batch_size=batch_size, normalize=normalize)
    embs = embs.astype(np.float32) if store_dtype == "float32" else embs.astype(np.float16)

    # Build DataFrame
    chunk_df = pd.DataFrame({
        "review_id": ids,
        "review_text_embeddings": [row.tolist() for row in embs]
    })

    # Append to Parquet
    if not os.path.exists(out_parquet_path):
        # Write new file
        chunk_df.to_parquet(out_parquet_path, engine="pyarrow", index=False, compression="zstd")
    else:
        # Append
        existing = pq.read_table(out_parquet_path).to_pandas()
        combined = pd.concat([existing, chunk_df], ignore_index=True)
        combined.to_parquet(out_parquet_path, engine="pyarrow", index=False, compression="zstd")

    print(f"✅ Batch {batch_idx} [{start}:{end}] saved to {out_parquet_path}")

In [ ]:
# Total number of batches
num_batches = math.ceil(len(all_reviews_labeled_df) / 2000)

# Run batch 0
embed_review_texts_batch(all_reviews_labeled_df, "./review_text_embeddings_0.parquet", batch_idx=0)

Embedding (Qwen3-Emb-0.6B): 100%|██████████| 32/32 [28:04<00:00, 52.63s/it]


✅ Batch 0 [0:2000] saved to ./review_text_embeddings_0.parquet


In [ ]:
embed_review_texts_batch(all_reviews_labeled_df, "./review_text_embeddings_1.parquet", batch_idx=1)

Embedding (Qwen3-Emb-0.6B): 100%|██████████| 32/32 [26:52<00:00, 50.40s/it]


✅ Batch 1 [2000:4000] saved to ./review_text_embeddings.parquet


In [ ]:
embed_review_texts_batch(all_reviews_labeled_df, "./review_text_embeddings_2.parquet", batch_idx=2)

Embedding (Qwen3-Emb-0.6B): 100%|██████████| 32/32 [2:21:33<00:00, 265.42s/it]   


✅ Batch 2 [4000:6000] saved to ./review_text_embeddings_2.parquet


In [ ]:
MAX_LEN  = 256
BATCH    = 64

def embed_categories_batch(
    df: pd.DataFrame,
    out_parquet_path: str,
    batch_idx: int,
    chunk_rows: int = 2000,
    max_len: int = MAX_LEN,
    batch_size: int = BATCH,
    normalize: bool = True,
    store_dtype: str = "float32",
):
    """
    Embed ONE batch of category lists and save to a Parquet file.

    Args:
        df : DataFrame with 'review_id' and 'categories' (list[str])
        out_parquet_path : Path to Parquet file
        batch_idx : Which batch index to embed (0-based)
        chunk_rows : Number of rows per batch
    """
    assert "review_id" in df.columns and "category" in df.columns, \
        "DataFrame must have 'review_id' and 'category' columns."

    N = len(df)
    start = batch_idx * chunk_rows
    end = min(start + chunk_rows, N)
    if start >= N:
        raise IndexError(f"Batch {batch_idx} out of range. Max = {(N-1)//chunk_rows}")

    # Slice in order
    sl = df.iloc[start:end]
    cat_texts = sl["category"].apply(lambda x: ", ".join(x) if isinstance(x, (list, tuple)) else str(x)).tolist()
    ids = sl["review_id"].tolist()

    # Embed category text
    embs = embed_texts(cat_texts, max_len=max_len, batch_size=batch_size, normalize=normalize)
    embs = embs.astype(np.float32) if store_dtype == "float32" else embs.astype(np.float16)

    # Build DataFrame
    chunk_df = pd.DataFrame({
        "review_id": ids,
        "categories_embeddings": [row.tolist() for row in embs]
    })

    # Append or write new Parquet
    if not os.path.exists(out_parquet_path):
        chunk_df.to_parquet(out_parquet_path, engine="pyarrow", index=False, compression="zstd")
    else:
        existing = pq.read_table(out_parquet_path).to_pandas()
        combined = pd.concat([existing, chunk_df], ignore_index=True)
        combined.to_parquet(out_parquet_path, engine="pyarrow", index=False, compression="zstd")

    print(f"✅ Categories batch {batch_idx} [{start}:{end}] saved to {out_parquet_path}")

In [ ]:
embed_categories_batch(
    all_reviews_labeled_df,
    out_parquet_path="./categories_embeddings_0.parquet",
    batch_idx=0,
    chunk_rows=2000,
    max_len=MAX_LEN,
    batch_size=BATCH,
    normalize=True,
    store_dtype="float32"
)

Embedding (Qwen3-Emb-0.6B): 100%|██████████| 32/32 [05:02<00:00,  9.46s/it]


✅ Categories batch 0 [0:2000] saved to ./categories_embeddings_0.parquet


In [ ]:
# Batch 1
# embed_categories_batch(
#     all_reviews_labeled_df,
#     out_parquet_path="./categories_embeddings_1.parquet",
#     batch_idx=1,
#     chunk_rows=2000,
#     max_len=MAX_LEN,
#     batch_size=BATCH,
#     normalize=True,
#     store_dtype="float32"
# )

# # Batch 2
# embed_categories_batch(
#     all_reviews_labeled_df,
#     out_parquet_path="./categories_embeddings_2.parquet",
#     batch_idx=2,
#     chunk_rows=2000,
#     max_len=MAX_LEN,
#     batch_size=BATCH,
#     normalize=True,
#     store_dtype="float32"
# )

# Batch 3
embed_categories_batch(
    all_reviews_labeled_df,
    out_parquet_path="./categories_embeddings_3.parquet",
    batch_idx=3,
    chunk_rows=2000,
    max_len=MAX_LEN,
    batch_size=BATCH,
    normalize=True,
    store_dtype="float32"
)

# Batch 4
embed_categories_batch(
    all_reviews_labeled_df,
    out_parquet_path="./categories_embeddings_4.parquet",
    batch_idx=4,
    chunk_rows=2000,
    max_len=MAX_LEN,
    batch_size=BATCH,
    normalize=True,
    store_dtype="float32"
)

# Batch 5
embed_categories_batch(
    all_reviews_labeled_df,
    out_parquet_path="./categories_embeddings_5.parquet",
    batch_idx=5,
    chunk_rows=2000,
    max_len=MAX_LEN,
    batch_size=BATCH,
    normalize=True,
    store_dtype="float32"
)

Embedding (Qwen3-Emb-0.6B): 100%|██████████| 32/32 [07:15<00:00, 13.62s/it]


✅ Categories batch 3 [6000:8000] saved to ./categories_embeddings_3.parquet


Embedding (Qwen3-Emb-0.6B): 100%|██████████| 32/32 [03:54<00:00,  7.34s/it]


✅ Categories batch 4 [8000:10000] saved to ./categories_embeddings_4.parquet


Embedding (Qwen3-Emb-0.6B): 100%|██████████| 30/30 [00:48<00:00,  1.62s/it]


✅ Categories batch 5 [10000:11920] saved to ./categories_embeddings_5.parquet


In [16]:
all_reviews_labeled_df = pd.read_csv('all_reviews_with_labels_normalised.csv')

In [2]:
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
MODEL_NAME = "gpt-4o-mini"

SENTIMENT_PROMPT_TEMPLATE = """\
You are a strict sentiment rater for short customer reviews.

TASK:
- Read the review text.
- Output a single numeric sentiment score in the range [-1.0, 1.0]:
  -1.0 = extremely negative
   0.0 = neutral/mixed/unclear
  +1.0 = extremely positive

OUTPUT RULES:
- Return ONLY compact JSON with a single key 'sentiment_score'.
- No markdown, no extra text, no explanations.
- Example valid output: {{"sentiment_score": -0.42}}

REVIEW:
{review_text}
""".strip()

def classify_sentiment(review_text: str) -> dict:
    """Return {"sentiment_score": float in [-1,1]} with JSON-only response."""
    if not isinstance(review_text, str) or len(review_text.strip()) == 0:
        return {"sentiment_score": 0.0}

    prompt = SENTIMENT_PROMPT_TEMPLATE.format(review_text=review_text.strip())

    # basic retry with exponential backoff
    for attempt in range(5):
        try:
            resp = client.chat.completions.create(
                model=MODEL_NAME,
                messages=[{"role": "user", "content": prompt}],
                response_format={"type": "json_object"}
            )
            content = resp.choices[0].message.content
            data = json.loads(content)

            # validate / clamp
            score = float(data.get("sentiment_score", 0.0))
            score = max(-1.0, min(1.0, score))
            return {"sentiment_score": score}

        except Exception as e:
            # backoff
            sleep_s = 2 ** attempt
            time.sleep(sleep_s)
            last_err = e

    # fallback on persistent failure
    return {"sentiment_score": 0.0}

In [12]:
def gpt_sentiment_batch(
    df: pd.DataFrame,
    batch_idx: int,
    text_col: str = "review_text",
    id_col: str = "review_id",
    out_prefix: str = "review_sentiment_scores",
    chunk_size: int = 2000,
):
    """
    Run GPT sentiment analysis for ONE batch only (batch_idx).
    Saves results to: {out_prefix}_{batch_idx}.parquet

    Each file has two columns: review_id, sentiment_score
    """
    assert id_col in df.columns and text_col in df.columns, "Missing required columns."

    N = len(df)
    start = batch_idx * chunk_size
    end = min(start + chunk_size, N)

    if start >= N:
        raise IndexError(f"Batch {batch_idx} out of range. Max batch index = {(N-1)//chunk_size}")

    sub = df.iloc[start:end]
    ids = sub[id_col].tolist()
    texts = sub[text_col].fillna("").astype(str).tolist()

    rows = []
    for i in tqdm(range(len(texts)), desc=f"Batch {batch_idx} (reviews)"):
        result = classify_sentiment(texts[i])  # your GPT call
        rows.append((ids[i], result["sentiment_score"]))

    out_df = pd.DataFrame(rows, columns=[id_col, "sentiment_score"])
    out_path = f"{out_prefix}_{batch_idx}.parquet"
    out_df.to_parquet(out_path, index=False)
    print(f"✅ Saved batch {batch_idx} [{start}:{end}] → {out_path}")

    return out_df

In [13]:
gpt_sentiment_batch(all_reviews_labeled_df, batch_idx=0, text_col="review_text")

Batch 0 (reviews): 100%|██████████| 2000/2000 [24:41<00:00,  1.35it/s]

✅ Saved batch 0 [0:2000] → review_sentiment_scores_0.parquet


,review_id,sentiment_score
0,1001,0.80
1,1002,-0.85
2,1003,1.00
3,1004,-1.00
4,1005,-1.00
...,...,...
1995,2996,0.80
1996,2997,0.90
1997,2998,0.60
1998,2999,1.00


In [ ]:
# # Batch 1
# gpt_sentiment_batch(all_reviews_labeled_df, batch_idx=1, text_col="review_text")

# # Batch 2
# gpt_sentiment_batch(all_reviews_labeled_df, batch_idx=2, text_col="review_text")

# # Batch 3
# gpt_sentiment_batch(all_reviews_labeled_df, batch_idx=3, text_col="review_text")

# # Batch 4
# gpt_sentiment_batch(all_reviews_labeled_df, batch_idx=4, text_col="review_text")

# # Batch 5
# gpt_sentiment_batch(all_reviews_labeled_df, batch_idx=5, text_col="review_text")

Batch 1 (reviews): 100%|██████████| 2000/2000 [25:11<00:00,  1.32it/s]


✅ Saved batch 1 [2000:4000] → review_sentiment_scores_1.parquet


Batch 2 (reviews): 100%|██████████| 2000/2000 [22:28<00:00,  1.48it/s] 


✅ Saved batch 2 [4000:6000] → review_sentiment_scores_2.parquet


Batch 3 (reviews): 100%|██████████| 2000/2000 [23:14<00:00,  1.43it/s]


...

Batch 4 (reviews): 100%|██████████| 2000/2000 [22:42<00:00,  1.47it/s]


✅ Saved batch 4 [8000:10000] → review_sentiment_scores_4.parquet


Batch 5 (reviews): 100%|██████████| 1920/1920 [3:24:49<00:00,  6.40s/it]  

✅ Saved batch 5 [10000:11920] → review_sentiment_scores_5.parquet


,review_id,sentiment_score
0,17458,0.9
1,17459,0.9
2,17461,1.0
3,17463,0.9
4,17464,1.0
...,...,...
1915,20600,-0.5
1916,20601,0.8
1917,20609,0.0
1918,20610,0.0


### Generating `average_score`, `has_phone`, `has_link` and `has_email` features

In [ ]:
# 1. Compute average_score per business
business_avg = all_reviews_labeled_df.groupby("business_name")["rating"].mean().rename("average_rating")

# Merge into main df
all_reviews_labeled_df = all_reviews_labeled_df.merge(business_avg, on="business_name", how="left")

# 2. Regex helpers for features
def has_phone(text: str) -> int:
    # simple phone detection: (123) 456-7890 or 123-456-7890 or +65 1234 5678
    return int(bool(re.search(r"(\+?\d{1,3}[\s-]?)?\(?\d{2,4}\)?[\s-]?\d{3,4}[\s-]?\d{3,4}", str(text))))

def has_link(text: str) -> int:
    return int(bool(re.search(r"http[s]?://|www\.", str(text).lower())))

def has_email(text: str) -> int:
    return int(bool(re.search(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", str(text))))

# 3. Apply to create new columns
all_reviews_labeled_df["has_phone"] = all_reviews_labeled_df["review_text"].apply(has_phone)
all_reviews_labeled_df["has_link"]  = all_reviews_labeled_df["review_text"].apply(has_link)
all_reviews_labeled_df["has_email"] = all_reviews_labeled_df["review_text"].apply(has_email)

In [ ]:
all_reviews_labeled_df["has_phone"].value_counts()

has_phone
0    11918
1        2
Name: count, dtype: int64

In [ ]:
all_reviews_labeled_df["has_link"].value_counts()

has_link
0    11920
Name: count, dtype: int64

In [ ]:
all_reviews_labeled_df["has_email"].value_counts()

has_email
0    11920
Name: count, dtype: int64

The `has_phone`, `has_link` and `has_email` features are not useful in predicting the class of the review given that most values do not have phone numbers, links or emails. The little variance will mean that the model will ignore the feature or treat it as noise i.e. the feature will not be helpful in predicting the class of the review.

In [ ]:
all_reviews_labeled_df["rating_discrepancy"] = abs(all_reviews_labeled_df["rating"] - all_reviews_labeled_df["average_rating"])

In [ ]:
all_reviews_labeled_df.to_csv("all_reviews_with_labels_and_features.csv", index=False)

## Combining parquet files

In [20]:
def combine_parquet_shards(prefix: str, start: int, end: int, id_col: str, out_path: str) -> pd.DataFrame:
    """
    Combine parquet shards like f'{prefix}_{i}.parquet' for i in [start, end],
    de-duplicate on `id_col`, sort by `id_col`, and save to `out_path`.
    """
    dfs = []
    for i in range(start, end + 1):
        fp = Path(f"{prefix}_{i}.parquet")
        if fp.exists():
            dfs.append(pd.read_parquet(fp))
        else:
            print(f"⚠️ Skipping missing file: {fp}")

    if not dfs:
        raise FileNotFoundError(f"No shards found for prefix '{prefix}' in range [{start}, {end}]")

    combined = pd.concat(dfs, ignore_index=True)

    # Ensure the id column exists
    if id_col not in combined.columns:
        raise KeyError(f"'{id_col}' not found in combined DataFrame columns: {combined.columns.tolist()}")

    # Drop duplicate ids (keep the last occurrence) and sort
    combined = (
        combined
        .drop_duplicates(subset=[id_col], keep="last")
        .sort_values(by=id_col)
        .reset_index(drop=True)
    )

    # Save combined file
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    combined.to_parquet(out_path, index=False)
    print(f"✅ Saved: {out_path}  | shape={combined.shape}")
    return combined


# --- Combine category embeddings shards (0..5) ---
categories_embeddings_all = combine_parquet_shards(
    prefix="categories_embeddings",
    start=0,
    end=5,
    id_col="review_id",
    out_path="combined/categories_embeddings_all.parquet",
)

# --- Combine review sentiment score shards (0..5) ---
review_sentiment_scores_all = combine_parquet_shards(
    prefix="review_sentiment_scores",
    start=0,
    end=5,
    id_col="review_id",
    out_path="combined/review_sentiment_scores_all.parquet",
)

# --- Combine review text embeddings shards (0..5) ---
review_text_embeddings_all = combine_parquet_shards(
    prefix="review_text_embeddings",
    start=0,
    end=5,
    id_col="review_id",
    out_path="combined/review_text_embeddings_all.parquet",
)

✅ Saved: combined/categories_embeddings_all.parquet  | shape=(11920, 2)
✅ Saved: combined/review_sentiment_scores_all.parquet  | shape=(11920, 2)
✅ Saved: combined/review_text_embeddings_all.parquet  | shape=(11920, 2)


In [21]:
all_reviews_with_labels_and_features = pd.read_csv("all_reviews_with_labels_and_features.csv")

# Load the combined sentiment scores
review_sentiment_scores_all = pd.read_parquet("combined/review_sentiment_scores_all.parquet")

# Merge on 'review_id'
all_with_sentiment = all_reviews_with_labels_and_features.merge(
    review_sentiment_scores_all,
    on="review_id",
    how="left"   # keep all rows from main df, even if sentiment is missing
)

# Save the enriched dataframe
all_with_sentiment.to_parquet("combined/all_reviews_with_labels_and_features.parquet", index=False)

print("✅ Final dataframe shape:", all_with_sentiment.shape)
print("✅ Saved: combined/all_reviews_with_labels_and_features.parquet")

✅ Final dataframe shape: (11920, 21)
✅ Saved: combined/all_reviews_with_labels_and_features.parquet


# 2. Model Architecture and Training

* Input = embeddings + numeric features

* Hidden Layers: Dense(256) → ReLU → Dropout(0.3) → Dense(128) → ReLU

* Output Layer: 3 logits (is_ad, is_relevant, is_rant)

Loss & optimization:

* Focal Loss (multi-label)

* Adam optimizer (lr=1e-3) with weight decay

In [25]:
# ---------------------------
# 1. Load and prepare dataset
# ---------------------------
df = pd.read_parquet("combined/all_reviews_with_labels_and_features.parquet")
cat_emb = pd.read_parquet("combined/categories_embeddings_all.parquet")
text_emb = pd.read_parquet("combined/review_text_embeddings_all.parquet")

# Merge embeddings into main df
df = df.merge(cat_emb, on="review_id", how="left").merge(text_emb, on="review_id", how="left")

# Features to use
feature_cols = [
    "rating", "has_photo", "average_score", "average_rating", "rating_discrepancy", "sentiment_score"
]
# Embedding cols
embedding_cols = [c for c in df.columns if c.startswith("embedding_")]

X = df[feature_cols + embedding_cols].fillna(0).values.astype(np.float32)
y = df[["is_ad", "is_relevant", "is_rant"]].values.astype(np.float32)

# Standardize numeric features (not embeddings)
scaler = StandardScaler()
X[:, :len(feature_cols)] = scaler.fit_transform(X[:, :len(feature_cols)])

In [27]:
# -----------------------------------------------
# 2) Multi-label stratified split (if available)
# -----------------------------------------------
def multilabel_train_val_split(X, y, test_size=0.2, random_state=42):
    """
    Try iterative multi-label stratification. If the package isn't available
    or labels are too sparse, fall back to a plain random split.
    """
    # Check sparsity: we need at least 2 positives AND 2 negatives per label
    pos_counts = y.sum(axis=0)
    neg_counts = y.shape[0] - pos_counts
    ok_per_label = (pos_counts >= 2) & (neg_counts >= 2)

    try:
        from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
        if ok_per_label.all():
            msss = MultilabelStratifiedShuffleSplit(
                n_splits=1, test_size=test_size, random_state=random_state
            )
            train_idx, val_idx = next(msss.split(X, y))
            return X[train_idx], X[val_idx], y[train_idx], y[val_idx], True
        else:
            print("⚠️ Some labels too sparse for stratification "
                  f"(pos={pos_counts.tolist()}, neg={neg_counts.tolist()}). Using random split.")
            raise RuntimeError("sparse_labels")
    except Exception:
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=test_size, random_state=random_state, shuffle=True
        )
        return X_train, X_val, y_train, y_val, False

X_train, X_val, y_train, y_val, used_strat = multilabel_train_val_split(X, y)

train_ds = TensorDataset(torch.tensor(X_train), torch.tensor(y_train).float())
val_ds   = TensorDataset(torch.tensor(X_val),   torch.tensor(y_val).float())

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False)

In [28]:
# ---------------------------
# 3) Define neural net model
# ---------------------------
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=256, output_dim=3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, output_dim)  # logits
        )
    def forward(self, x):
        return self.net(x)

model = MLPClassifier(input_dim=X.shape[1], hidden_dim=256, output_dim=y.shape[1])

# 3. Handling Class Imbalance

* Computed positive weights for rare labels.

* Applied WeightedRandomSampler to oversample minority classes.

* Used focal Loss to focus learning on hard/rare cases.

In [29]:
# ---------------------------
# 4) Handle class imbalance
# ---------------------------
# Weighted BCE per-output using inverse prevalence on the TRAIN SET
pos = y_train.sum(axis=0).astype(np.float32)
neg = y_train.shape[0] - pos
pos_weight = torch.tensor(neg / (pos + 1e-6), dtype=torch.float32)  # shape [n_labels]

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 4. Threshold Tuning

Per-label threshold tuning using validation precision-recall curves. Prevented “over-positive” bias (e.g., is_ad → high recall but low precision).

In [32]:
from sklearn.metrics import f1_score, precision_recall_curve

def tune_thresholds(y_true, y_prob):
    """
    Per-label threshold search maximizing F1 on the validation set.
    Returns array of best thresholds per label.
    """
    n_labels = y_true.shape[1]
    best = np.zeros(n_labels)
    for j in range(n_labels):
        prec, rec, thr = precision_recall_curve(y_true[:, j], y_prob[:, j])
        # Map PR points to F1; thr length = len(prec)-1; use those
        f1s = 2 * prec[:-1] * rec[:-1] / (prec[:-1] + rec[:-1] + 1e-12)
        if len(f1s) == 0:
            best[j] = 0.5
        else:
            best[j] = max(0.05, min(0.95, thr[np.argmax(f1s)]))  # clamp to reasonable range
    return best

In [33]:
class FocalLossMultiLabel(nn.Module):
    """
    Multi-label focal loss with per-label alpha (class balance) and gamma.
    y_pred: logits (before sigmoid), y_true: {0,1}
    """
    def __init__(self, alpha=None, gamma=2.0, reduction="mean"):
        super().__init__()
        self.alpha = alpha  # torch.tensor shape [n_labels] or None
        self.gamma = gamma
        self.reduction = reduction
        self.bce = nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, logits, targets):
        bce = self.bce(logits, targets)
        p = torch.sigmoid(logits)
        pt = p*targets + (1-p)*(1-targets)       # p_t
        focal = (1-pt).pow(self.gamma) * bce
        if self.alpha is not None:
            focal = focal * self.alpha  # broadcasts per-label alpha
        if self.reduction == "mean":
            return focal.mean()
        elif self.reduction == "sum":
            return focal.sum()
        return focal

In [36]:
# ---------------------------
# 5) Train
# ---------------------------
# y_train, y_val from your split (0/1 floats)
n_labels = y_train.shape[1]

# ---- Balanced sampler (sample minority positives more often) ----
# weight per sample = mean over labels of (y*pos_w + (1-y)*neg_w)
pos_counts = y_train.sum(axis=0)
neg_counts = y_train.shape[0] - pos_counts
pos_w = (neg_counts / (pos_counts + 1e-6)).astype(np.float32)
pos_w = np.clip(pos_w, 1.0, 5.0)   # cap to avoid over-correction
neg_w = np.ones_like(pos_w, dtype=np.float32)

sample_w = (y_train * pos_w + (1 - y_train) * neg_w).mean(axis=1)
sampler = WeightedRandomSampler(weights=torch.tensor(sample_w), num_samples=len(sample_w), replacement=True)

train_loader = DataLoader(train_ds, batch_size=64, sampler=sampler)
val_loader   = DataLoader(val_ds,   batch_size=64, shuffle=False)

# ---- Focal loss with alpha per-label (class balance) ----
alpha = torch.tensor(pos_w, dtype=torch.float32)  # larger alpha for rare labels
criterion = FocalLossMultiLabel(alpha=alpha, gamma=2.0)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)  # add L2
best_macro_f1, patience, bad = 0.0, 3, 0
best_state = None

for epoch in range(1, 30 + 1):
    # train
    model.train()
    total = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()
        total += loss.item() * xb.size(0)
    train_loss = total / len(train_ds)

    # validate (use 0.5 thresholds first to guide early stopping)
    model.eval()
    probs_list, true_list = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            logits = model(xb)
            probs = torch.sigmoid(logits).cpu().numpy()
            probs_list.append(probs); true_list.append(yb.numpy())
    val_probs = np.vstack(probs_list)
    val_true  = np.vstack(true_list)
    val_pred05 = (val_probs >= 0.5).astype(int)
    macro_f1 = f1_score(val_true, val_pred05, average="macro")

    print(f"Epoch {epoch:02d} | train_loss={train_loss:.4f} | macroF1@0.5={macro_f1:.3f}")

    # early stopping on macro-F1
    if macro_f1 > best_macro_f1 + 1e-4:
        best_macro_f1 = macro_f1
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        bad = 0
    else:
        bad += 1
        if bad >= patience:
            print("Early stopping.")
            break

# load best
if best_state is not None:
    model.load_state_dict({k: v.to(device) for k, v in best_state.items()})

# ---- Per-label threshold tuning on validation set ----
model.eval()
probs_list, true_list = [], []
with torch.no_grad():
    for xb, yb in val_loader:
        xb = xb.to(device)
        logits = model(xb)
        probs_list.append(torch.sigmoid(logits).cpu().numpy())
        true_list.append(yb.numpy())
val_probs = np.vstack(probs_list)
val_true  = np.vstack(true_list)

best_thr = tune_thresholds(val_true, val_probs)
print("Best thresholds per label:", best_thr)

val_pred = (val_probs >= best_thr).astype(int)
print(classification_report(val_true, val_pred, target_names=["is_ad","is_relevant","is_rant"]))

Epoch 01 | train_loss=0.2374 | macroF1@0.5=0.527
Epoch 02 | train_loss=0.2151 | macroF1@0.5=0.528
Epoch 03 | train_loss=0.2146 | macroF1@0.5=0.524
Epoch 04 | train_loss=0.2099 | macroF1@0.5=0.528
Epoch 05 | train_loss=0.2088 | macroF1@0.5=0.529
Epoch 06 | train_loss=0.2066 | macroF1@0.5=0.524
Epoch 07 | train_loss=0.2026 | macroF1@0.5=0.528
Epoch 08 | train_loss=0.2016 | macroF1@0.5=0.528
Early stopping.
Best thresholds per label: [0.27438629 0.58857948 0.52259874]
              precision    recall  f1-score   support

       is_ad       0.04      0.74      0.07        66
 is_relevant       0.96      1.00      0.98      2288
     is_rant       0.51      0.75      0.61       167

   micro avg       0.62      0.98      0.76      2521
   macro avg       0.50      0.83      0.55      2521
weighted avg       0.91      0.98      0.93      2521
 samples avg       0.67      0.95      0.76      2521



c:\Users\teomi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
